In [1]:
import sys
sys.path.append('../../')

from os import listdir

from hdf5storage import loadmat, savemat 
import numpy as np 
from matplotlib import pyplot as plt

from scipy import signal 
from hnlpy.timeop import makefiltersos
from hnlpy.timeop import narrowfilter 
from scipy.signal import sosfiltfilt
from scipy.signal import hilbert
from scipy.signal import savgol_filter
from scipy.fftpack import fft
from scipy import stats # for zscoring

from sklearn.covariance import GraphicalLassoCV

from numpy.polynomial.polynomial import polyfit, polyval

import matplotlib.pyplot as plt

/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, 

In [4]:
def avref(datax):
	ref = np.mean(datax,axis = 1)
	refmat = np.tile(ref,(np.shape(datax)[1],1))
	datax = datax - np.transpose(refmat)
	return datax

In [29]:
# build the narrow band filters
srnew = 200 
downsample = 10 
# passbands = [[1,3],[3.5,6.5],[7,10],[10.5,13.5],[14 ,20],[21,29],[30,49.5]]
# stopbands = [[0.75,3.25],[3.25,6.75],[6.75,10.25],[10.25,13.75],[13.75,20.5],[20.5,29.5],[29.5,50]]
passbands = [[3.5,6.5],[7,10],[10.5,13.5],[14 ,20],[21,29]]
stopbands = [[3.25,6.75],[6.75,10.25],[10.25,13.75],[13.75,20.5],[20.5,29.5]]
# bandlabels = ['Delta','Theta','Alpha','Mu','Beta1','Beta2','Gamma']
bandlabels = ['Theta','Alpha','Mu','Beta1','Beta2']
esos = dict()
for j in range(len(passbands)):
    esos[j],w,h = narrowfilter(passbands[j],stopbands[j],samplingrate = srnew)
    #plt.plot(w[1:500],20*np.log10(np.abs(h[1:500])))


In [8]:
ncv = 5
pc_lasso = GraphicalLassoCV(cv=ncv, n_jobs=-10)

In [9]:
# organize file in time sequence
filedates=[20220713,20220721,20220804,20220808,20220810,20220811,20220815,20220816,20221003,2022100401,2022100402,20221005]
numSes=len(filedates)

In [30]:
# make a low pass filter @ 50 Hz 
sr=2000
sos_low, w,h = makefiltersos(sr,50,55)

In [ ]:
prec_all=np.zeros((12,2,12,5,896,896))
for ses in range(2):
    for subj in range(2):
        for tr in range(12):
                filename='subj'+str(subj+1)+'_tr_'+str(tr+1)+'.mat'
                outdict=loadmat('/ssd/zhibin/Cleaned_sourcedata/cortical_source_data/'+str(filedates[ses])+'/'+filename) # condition not sorted
                agr_source_data=outdict['agr_source_data']
                trialdata = avref(agr_source_data) 
                trialdata = sosfiltfilt(sos_low,trialdata,axis=0)
                nsamp=np.shape(trialdata)[0]
                trialdatanew = trialdata[range(0,nsamp,downsample),:]
                for freq in range(5):
                    filtdata = sosfiltfilt(esos[freq],trialdatanew,axis = 0,padtype ='odd')
                    hilbertdata = hilbert(filtdata,axis = 0)
                    # Combine real and imaginary parts
                    sourceDataReal = np.concatenate((np.real(hilbertdata), np.imag(hilbertdata)), axis=1)
                    # Normalize data
                    sourceDataReal = sourceDataReal * (1 / np.mean(np.abs(sourceDataReal)))
                    pc_lasso.fit(sourceDataReal)
                    precision = pc_lasso.precision_
                    y = np.diag(precision)
                    yy = np.outer(y,y)
                    yy = np.sqrt(yy)
                    prec_all[ses,subj,tr,freq,:,:] = precision/yy
